In [1]:
from lightning_wrapper import *

In [2]:
BATCH_SIZE = 64

In [3]:
class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [4]:
def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        y = y.view(-1, 1)
        loss = nn.MSELoss()(y_hat, y)
        self.log("train_loss", loss)
        return loss
    
def test_step(self, batch, batch_idx):
    # this is the test loop
    x, y = batch
    y_hat = self.model(x)
    y = y.view(-1, 1)
    loss = F.mse_loss(y_hat, y)
    self.log("test_loss", loss)

def configure_optimizers(self):
    optimizer = optim.Adam(self.parameters(), lr=1e-3)
    return optimizer

def validation_step(self, batch):
    x, y = batch
    y_hat = self.model(x)
    y = y.view(-1, 1)
    loss = nn.MSELoss()(y_hat, y)
    self.log("val_loss", loss)
    return loss

In [5]:
train_dataloader, val_dataloader, test_dataloader = get_diabetes_data(batch_size=BATCH_SIZE)

In [6]:
feature_size = train_dataloader.dataset.tensors[0].shape[1]
torchinfo.summary(ANN(), input_size=(BATCH_SIZE, feature_size))

Layer (type:depth-idx)                   Output Shape              Param #
ANN                                      [64, 1]                   --
├─Linear: 1-1                            [64, 64]                  704
├─ReLU: 1-2                              [64, 64]                  --
├─Dropout: 1-3                           [64, 64]                  --
├─Linear: 1-4                            [64, 128]                 8,320
├─ReLU: 1-5                              [64, 128]                 --
├─Dropout: 1-6                           [64, 128]                 --
├─Linear: 1-7                            [64, 1]                   129
Total params: 9,153
Trainable params: 9,153
Non-trainable params: 0
Total mult-adds (M): 0.59
Input size (MB): 0.00
Forward/backward pass size (MB): 0.10
Params size (MB): 0.04
Estimated Total Size (MB): 0.14

In [7]:
lightning_model = LightningModel(ANN, training_step, test_step, validation_step, configure_optimizers)

In [8]:
trainer = l.Trainer(callbacks=[EarlyStopping(monitor="val_loss", mode="min")], max_epochs=100, default_root_dir="events")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
print(list(lightning_model.parameters()))

[]


In [10]:
trainer.fit(lightning_model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

ValueError: optimizer got an empty parameter list